In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Riemann Sums, Integration, and Messy Data

**Main problem.** You run a small lab analyzing ice cores from the Good Humor Glacier in Greenland. You keep the sample in a large $2\text{m}\times 2\text{m}\times 3\text{m}$ freezer. To make sure it is keeping the samples properly cold, you need to know its average temperature.
You task your grad student with measuring this and send them in with a digital thermometer. 

They return with a data set of 60 temperature readings and report and an average reading of $20.565^\circ$. You take one look at the data and know immediately it is bunk. How? 

### Learning Objectives

At the conclusion of this workshop, you should be able to:
  - write a Riemann sum estimation of double and triple integrals using $\Sigma$-notation
  - use `numpy` to estimate an integral using a number of different schema
  - use `pandas` to view, sort, and do some simple filtering of a data set
  - explore a few regression models via `scikit-learn`

Recall that a Riemann sum for a double integral has the form 

$$\iint_D f \,dA \approx \sum_{i,j} f(x_{ij}^*,y_{ij}^*) \Delta A_{ij}$$

where $(x_{ij}^*,y_{ij}^*)$ is a point in the rectangle with indices $i,j$ and $\Delta A_{ij}$ is its area.

## Northeast Rule

In the case that our domain is the rectangle $[a,b]\times [c,d]$ is divided into $n^2$ subrectangles and we use the "northeast" corner of each rectangle as our sample point, this can be further specified with $\Delta x = \frac{b-a}{n}, \Delta y = \frac{d-c}{n}$,  

$$\sum_{i=1}^n \sum_{j=1}^n f\left( a + i \Delta x,c + j \Delta y \right) \Delta x \Delta y$$

We implement this as follows.

In [3]:
def northeast_rule(f,a,b,c,d,n):
    '''An approximation of the integral of f over domain (a,b) x (c,d) by n^2 rectangles using 
    the upper-right corner as the sample point.'''
    dx = (b-a) / n
    dy = (d-c) / n
    return sum([f(a + i * dx,c + j * dy) for i in range(1,n+1) for j in range(1,n+1)])*dx*dy

Now, we test it by approximating $$\int_0^1 \int_0^1 \left(x^2 + y^2 \right) \,dy\,dx.$$

**Quick Exercise**. What is the exact answer?

In [5]:
def p(x,y): return x**2 + y**2
    
northeast_rule(p,0,1,0,1,1000)

0.6676670000000015

## Exercise 1.
Implement the Riemann sum using the "southeast" corner as sample point.

In [6]:
def southeast_rule(f,a,b,c,d,n):
    '''An approximation of the integral of f over domain (a,b) x (c,d) by n^2 rectangles using 
    the lower-left corner as the sample point.'''
    ### BEGIN SOLUTION
    dx = (b-a) / n
    dy = (d-c) / n
    return sum([f(a + i * dx,c + j * dy) for i in range(1,n+1) for j in range(n)])*dx*dy
    ### END SOLUTION

In [8]:
southeast_rule(p,0,1,0,1,1000)

0.6666670000000005

## Exercise 2.

Now implement the sum with the middle point of the rectangle.

In [9]:
def midpoint_rule(f,a,b,c,d,n):
    '''An approximation of the integral of f over domain (a,b) x (c,d) by n^2 rectangles.'''
    ### BEGIN SOLUTION
    dx = (b-a) / n
    dy = (d-c) / n
    return sum([f(a + (i+1/2) * dx,c + (j+1/2) * dy) for i in range(n) for j in range(n)])*dx*dy
    ###END SOLUTION

In [11]:
midpoint_rule(p,0,1,0,1,1000)

0.6666665000000263

## Exercise 3.

To cut down on variables, in the above, both axes were divided into an equal number of subintervals, but of course, we can treat each axis independently. 

Rewrite the midpoint rule above to divide the region into $m\times n$ subrectangles.

In [9]:
def midpoint_rule_mn(f,a,b,c,d,m,n):
    '''An approximation of the integral of f over domain (a,b) x (c,d) by n^2 rectangles.'''
    ### BEGIN SOLUTION
    dx = (b-a) / m
    dy = (d-c) / n
    return sum([f(a + (i+1/2) * dx,c + (j+1/2) * dy) for i in range(m) for j in range(n)])*dx*dy
    ###END SOLUTION

<hr>

## Extension - Other methods

Recall that in one-variable calulus, we had higher-order methods of integration, such as the trapezoid rule.

In [3]:
def trapezoid_rule(f,a,b,n):
    '''Estimate the integral of f over the interval [a,b] using the trapezoid rule.'''
    dx = (b - a)/n
    return ((f(a) + f(b))/2 + sum([f(a + i*dx) for i in range(1,n)]))*dx

In [13]:
trapezoid_rule(lambda x: x**13,0,1,12900)

0.07142857793859678

### Questions

  - How would you extend the trapezoid rule to the 2- and 3-variable cases? 
  - What about Simpson's rule?
  
These don't have very cut-and-dried answers. 

### Modern Methods

Numerical integration is often referred to by its old-timey name *quadrature*. There are far more accurate and efficient methods than are exhibited here (feel free to explore the rabbit-hole of [Gaussian Quadrature](http://mathworld.wolfram.com/GaussianQuadrature.html)).

That said, you can implement them easily using the SciPy module, specifically the `quad`, `dblquad`, and `tplquad` functions. 

In addition to using advanced methods, these functions can handle non-rectangular domains by passing functions as limits of integration. 

In [14]:
from scipy.integrate import quad, dblquad, tplquad

In [19]:
quad(lambda x: x**3 + 3.5*x ,0,1)

(2.0, 2.220446049250313e-14)

The first number returned in the estimate of the integral. The second (usually quite small) number is the estimate of the absolute error.

Here is computation of the triple integral 

$$\int_0^1 \int_0^{1-x} \int_0^{x^2 + y^2} 9z\,dz\,dy\,dx $$

In [28]:
def g(x): return 1-x
def h(x,y): return x**2 + y**2
def k(z,y,x): return 9*z

tplquad(k,0,1,0,g,0,h)

(0.35, 4.9527977917301234e-14)